GESTURE-BASED ROCK-PAPER-SCISSORS GAME

In [3]:
pip install opencv-python mediapipe numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

def classify_gesture(landmarks):
    finger_tips = [8, 12, 16, 20]
    finger_fold_status = []
    for tip in finger_tips:
        if landmarks[tip][1] < landmarks[tip - 2][1]:
            finger_fold_status.append(1)
        else:
            finger_fold_status.append(0)
    
    # Rock: All fingers folded
    if sum(finger_fold_status) == 0:
        return "Rock"
    # Paper: All fingers open
    elif sum(finger_fold_status) == 4:
        return "Paper"
    # Scissors: Only index and middle open
    elif finger_fold_status == [1, 1, 0, 0]:
        return "Scissors"
    else:
        return "Unknown"

def decide_winner(gesture1, gesture2):
    if gesture1 == gesture2:
        return "Draw!"
    rules = {
        'Rock': 'Scissors',
        'Scissors': 'Paper',
        'Paper': 'Rock'
    }
    if rules.get(gesture1) == gesture2:
        return "Player 1 Wins!"
    else:
        return "Player 2 Wins!"

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    gestures = []
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            lm_list = []
            for lm in hand_landmarks.landmark:
                lm_list.append([int(lm.x * w), int(lm.y * h)])
            gesture = classify_gesture(lm_list)
            gestures.append(gesture)
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    if len(gestures) == 2:
        winner = decide_winner(gestures[0], gestures[1])
        cv2.putText(frame, f"P1: {gestures[0]}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.putText(frame, f"P2: {gestures[1]}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.putText(frame, winner, (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,0,255), 3)
    else:
        cv2.putText(frame, "Show 2 hands!", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,0), 2)

    cv2.imshow("Rock Paper Scissors", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
